# Generic N-D sweeps

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import lightlab.util.sweep as sUtil

In [ ]:
# Turn this on when executing interactively
livePlots = False

In [ ]:
class Plant():
    def __init__(self):
        self.x = 2
        self.y = 2
        
    def actuateX(self, newX, rounded=False):
        self.x = round(newX) if rounded else newX
        
    def actuateY(self, newY):
        self.y = newY
        
    def measure(self):
        return (np.sin(self.x * self.y/3), self.y / self.x)

## Simplest case

In [ ]:
p = Plant()

swpInX = sUtil.NdSweeper()
swpInX.addActuation('xAct', lambda x: p.actuateX(x, rounded=False), np.linspace(15, 25, 20))
swpInX.addMeasurement('mOne', lambda: p.measure()[0])
swpInX.setMonitorOptions(stdoutPrint=not livePlots, livePlot=livePlots)

swpInX.gather()
if not livePlots:
    swpInX.plot()

## Multiple measurements and a domain parser

In [ ]:
swpInX.addMeasurement('mTwo', lambda: p.measure()[1])
swpInX.addParser('xAct^4', lambda d: d['xAct']**4) # Non actuated x-axis
swpInX.setPlotOptions(xKey=('xAct', 'xAct^4'))
swpInX.setMonitorOptions(stdoutPrint=not livePlots, livePlot=livePlots)
swpInX.gather()
if not livePlots:
    swpInX.plot()

### Adding new parsers after the data has been gathered

In [ ]:
swpInX.addParser('ratio', lambda d: d['mOne'] / d['mTwo'])
swpInX.addParser('square', lambda d: d['mOne'] * d['mOne'])
swpInX.setPlotOptions(xKey=('ratio',), yKey='square')
swpInX.plot()

## Subsuming to 2D and progress server

In [ ]:
swpInY = sUtil.NdSweeper()
swpInY.addActuation('yAct', p.actuateY, np.linspace(1, 2, 15))

fullSwp = swpInX.subsume(swpInY)

fullSwp.addParser('norm', lambda x: (x['mOne'] + x['xAct'])**2)
fullSwp.setMonitorOptions(runServer=True, livePlot=livePlots, plotEvery=10, stdoutPrint=True)
fullSwp.setPlotOptions(xKey=('yAct'), yKey=('norm', 'ratio'))
fullSwp.gather()

In [ ]:
fullSwp.setPlotOptions(plType='curves')
_=fullSwp.plot()

In [ ]:
fullSwp.setPlotOptions(plType='surf')
_=fullSwp.plot()

## Using static data to compare subsequent sweeps

In [ ]:
p = Plant()

# First do a 1d sweep
swpA = sUtil.NdSweeper()
swpA.addActuation('xAct', lambda x: p.actuateX(x, rounded=False), np.linspace(15, 25, 100))
swpA.addMeasurement('mOne', lambda: p.measure()[0])
swpA.setMonitorOptions(stdoutPrint=False)
swpA.gather()
swpA.plot()
plt.title('Baseline sweep')

# Get its data
baseline = swpA.data['mOne']

# do a 2d sweep where each line is compared to the 1d line
swpB = swpA.copy()
swpB.reinitActuation()
# The order of these calls matters
swpB.addActuation('xAct', lambda x: p.actuateX(x, rounded=True), np.linspace(15, 25, 100))
swpB.addStaticData('baseline', baseline) 
swpB.addActuation('yAct', lambda y: p.actuateY(y), np.linspace(1, 3, 3))
swpB.addParser('difference', lambda d: d['mOne'] - d['baseline'])
swpB.setPlotOptions(xKey='xAct', yKey='difference')
swpB.gather()
swpB.plot()
plt.title('Final comparison sweep')